In [1]:
import gensim.downloader as api
dataset = api.load("text8")
import math
import numpy as np
print(api.info("text8"))
import h5py

{'num_records': 1701, 'record_format': 'list of str (tokens)', 'file_size': 33182058, 'reader_code': 'https://github.com/RaRe-Technologies/gensim-data/releases/download/text8/__init__.py', 'license': 'not found', 'description': 'First 100,000,000 bytes of plain text from Wikipedia. Used for testing purposes; see wiki-english-* for proper full Wikipedia datasets.', 'checksum': '68799af40b6bda07dfa47a32612e5364', 'file_name': 'text8.gz', 'read_more': ['http://mattmahoney.net/dc/textdata.html'], 'parts': 1}


In [2]:
import itertools
corpus = list(itertools.chain.from_iterable(dataset))

In [3]:
print(len(corpus))

17005207


In [4]:
class Vocabulary:
    word_frequency={}
    id2Word = []
    word2Id = {}
    size = 0;
    blocks = 0;
    block_size = 0;
    areIdsCaculated = False;
    
    def __init__(self):
        self.word_frequency={}
        self.id2Word = []
        self.word2Id = {}
    
    def setBlock(self, blocks,block_length):
        self.blocks = blocks
        self.block_size = block_length
        
    def get_contrained_ids(self,words,segment):
        id_list = []
        for word in words:
            try:
                word_id = self.word2Id[word]
                if(segment*self.block_size <= word_id < (segment+1)*self.block_size):
                    id_list.append(word_id)
            except KeyError:
                #print('keyError' + str(words))
                pass
        return id_list
    
    def filter(self,number):
        self.word_frequency = dict(filter(lambda y: y[1] >= number,self.word_frequency.items()))
        
    def calcLowerBoundMemory(self):
        return 4 * len(self.word_frequency) * len(self.word_frequency);
    
    def calcLowerBoundMemoryInGb(self):
        return 4 * len(self.word_frequency) * len(self.word_frequency) / (1000 * 1000 * 1000.0);
    
    def assignIds(self,offset=0):
        i = offset
        for k in self.word_frequency.keys():
            self.id2Word.append(k)
            self.word2Id.update({k:i})
            i = i+1
        self.areIdsCaculated = True
        self.word_frequency = {}
        
    def get_size(self):
        if(self.areIdsCaculated):
            return len(self.id2Word)
        else:
            return len(self.word_frequency)
        
    def getIds(self,words):
        if not isinstance(words, list):
            raise KeyError
        id_list = []
        for word in words:
            try:
                id_list.append(self.word2Id[word])
            except KeyError:
                print('keyError' + str(words))
                pass
        return id_list
    

# Pass Nr 1 Code

In [5]:
def build_vocab(corpus_local):
    v1 = Vocabulary()
    for word in corpus_local:
        try:
            v1.word_frequency[word]+=1
        except KeyError:
            v1.word_frequency[word]=1
    return v1
    

In [6]:
vocab2 = build_vocab(corpus)
vocab2.filter(1000)
vocab2.assignIds()
print(vocab2.getIds(['the']))

[11]


# Pass Nr 2 Code

Select witch Words to capture

In [7]:
def build_co_occurence2(corpus, vocab_local, window_size,name,amount_split):
    f = h5py.File("{name}.hdf5".format(name=name), "w")
    HDF_matrix = f.create_dataset("matrix", (len(vocab_local.word2Id),len(vocab_local.word2Id)), dtype='i')
    window_half = int(window_size/2.0)#also index of center word
    
    #co_occurence = np.zeros((len(vocab_local.word2Id),len(vocab_local.word2Id)), dtype=int)
    block_length = math.ceil(vocab_local.get_size()/amount_split)
    print(block_length)
    vocab_local.setBlock(amount_split,block_length)
    
    for x in range(0,amount_split):
        for y in range(0,amount_split):
            rest_block_x = vocab_local.get_size() - block_length * x
            rest_block_y = vocab_local.get_size() - block_length * y
            local_matrix = np.zeros((min(block_length,rest_block_x)
                                     ,min(block_length,rest_block_y)), dtype=int)
            
        
        
            #go over corpus
            rolling_ids = []
            for i in range(0,len(corpus)- (2 * window_half)):
                if(i % 1000000 == 0):
                    print('progress: {percent:7.3f} %'.format(percent=(i/float(len(corpus)-window_half)*100.0)))
                #set window and roll
                #look up all or just the last one
                window = corpus[i:i+window_size]
                
                if(i == 0):
                    for start in range(0,window_size):
                        new_ids = vocab_local.get_contrained_ids([window[start]],y)
                        rolling_ids.append(new_ids)
                else:
                    rolling_ids = rolling_ids[1:]
                    new_ids = vocab_local.get_contrained_ids([window[-1]],y)
                    rolling_ids.append(new_ids)
                    
                #do the capture                            
                center_word = window[window_half]
                if(i == 0):
                    #loops first positions and then ids
                    for start_index in range(0,window_half):
                        current_size_right = start_index + window_half + 1
                        cur_window = window[0:current_size_right]
                        start_ids = vocab_local.get_contrained_ids([window[start_index]],x)
                        
                        for context_index in range(0,len(cur_window)): 
                            context_ids = rolling_ids[context_index]
                            
                            for start_id in start_ids:
                                for context_id in context_ids:
                                    local_matrix[start_id % block_length][context_id % block_length]+= 1 
                
                feature_ids = vocab_local.get_contrained_ids([center_word],x)
                for window_index in range(0,len(window)):
                    for feature_id in feature_ids:
                        for context_id in rolling_ids[window_index]:
                           #print(vocab_local.id2Word[current_id])
                           local_matrix[feature_id % block_length][context_id % block_length]+= 1
                            
                if(i == len(corpus)- (2 * window_half) - 1):#the last normal
                    for end in range(0,window_half):
                        cur_window = window[end+1:]#cut index + 1 off
                        end_ids = vocab_local.get_contrained_ids([window[-(window_half-end)]],x)
                                                       
                        for context_index in range(0,len(cur_window)): 
                            context_ids = rolling_ids[end+1+context_index]
                            for end_id in end_ids:
                                for context_id in context_ids:
                                    local_matrix[end_id % block_length][context_id % block_length]+= 1
            print(len(HDF_matrix),len(HDF_matrix[0]))
            print(len(local_matrix),len(local_matrix))
            print(x*block_length,(x+1)*block_length,y*block_length,(y+1)*block_length)
            HDF_matrix[x*block_length:(x+1)*block_length,y*block_length:(y+1)*block_length] = local_matrix
                
            print('pass done:'+str(x*amount_split+y+1) +'of '+ str(amount_split * amount_split))
    return HDF_matrix

In [8]:
vocab2 = build_vocab(corpus)
vocab2.filter(100)
vocab2.assignIds()





print(len(vocab2.word2Id))
matrix2 = build_co_occurence2(corpus,vocab2,5,"test28",1)



11815
11815
progress:   0.000 %
progress:   5.881 %
progress:  11.761 %
progress:  17.642 %
progress:  23.522 %
progress:  29.403 %
progress:  35.283 %
progress:  41.164 %
progress:  47.044 %
progress:  52.925 %
progress:  58.806 %
progress:  64.686 %
progress:  70.567 %
progress:  76.447 %
progress:  82.328 %
progress:  88.208 %
progress:  94.089 %
progress:  99.969 %
11815 11815
11815 11815
0 11815 0 11815
pass done:1of 1


Test test test

In [9]:
import tensorflow as tf
vector_size = 100
#tf.compat.v1.disable_eager_execution()

vocab_size = vocab2.get_size()

weights = ((np.random.rand(vocab_size, vector_size) - 0.5))

context_weights = ((np.random.rand(vector_size,vocab_size) - 0.5))

bias = ((np.random.rand(1,vocab_size) - 0.5))

context_bias = ((np.random.rand(vocab_size,1) - 0.5))


alpha = tf.constant(0.75,dtype=tf.dtypes.float32)
XMAX = tf.constant(10000.0,dtype=tf.dtypes.float32)


x = tf.Variable(0.5)



tf_bias = tf.Variable(initial_value=bias,dtype=tf.dtypes.float32)
tf_context_bias = tf.Variable(initial_value=context_bias,dtype=tf.dtypes.float32)
tf_weights =  tf.Variable(initial_value=weights,dtype=tf.dtypes.float32)
tf_context_weights = tf.Variable(initial_value=context_weights,dtype=tf.dtypes.float32)

tf_co_occurence = tf.convert_to_tensor(matrix2,dtype=tf.dtypes.float32)
epsilon = tf.constant(0.0001) * tf.ones((vocab_size,vocab_size),dtype=tf.dtypes.float32)

def loss():
    ones_symetrical = tf.ones((vocab_size,vocab_size), dtype=tf.dtypes.float32, name=None)
    bias_matrix = tf_bias * ones_symetrical + tf_context_bias * ones_symetrical
    
    weight_matrix = tf.matmul(tf_weights,tf_context_weights)
    log_X = tf.math.log(tf_co_occurence + epsilon)
    inner_sum = bias_matrix + weight_matrix - log_X
    squared_sum = tf.math.square(inner_sum)
    weighted_sum = cut_function2(tf_co_occurence) * squared_sum
    reduced = tf.math.reduce_sum(weighted_sum)
    return reduced
    
      
def cut_function2(value):
    clipped = tf.clip_by_value(value, clip_value_min = 0.0, clip_value_max=10000.0)
    return tf.pow(clipped / XMAX, alpha)


def compute_loss():
    y = tf.math.square(x)
    return y


tf.Tensor(376814.28, shape=(), dtype=float32)
Time to print it.


In [10]:
opt = tf.keras.optimizers.Adam(learning_rate=0.01)


In [13]:
for i in range(0,100):
    train = opt.minimize(loss, var_list=[tf_bias,tf_context_bias,tf_weights,tf_context_weights])
    print(loss().numpy())

347888.28
334276.94
320993.66
307876.25
294778.97
281572.78
268147.25
254414.88
240315.03
225819.9
210939.78
195727.62
180282.4
164750.39
149323.3
134232.62
119738.836
106114.8
93623.43
82491.13
72879.89
64863.11
58410.523
53387.453
49570.734
46680.297
44420.668
42524.15
40785.516
39080.906
37367.426
35666.297
34035.965
32543.395
31240.826
30151.908
29268.324
28554.861
27959.697
27426.207
26903.508
26353.814
25756.041
25105.912
24413.324
23698.055
22984.809
22298.387
21659.734
21083.197
20575.232
20134.555
19753.455
19419.973
19120.373
18841.57
18572.936
18307.4
18041.686
17775.84


KeyboardInterrupt: 

In [14]:
id = vocab2.getIds(['the'])
print(id)
print(tf_weights[id])
print(tf_weights[vocab2.getIds(['as'])])
print(tf_weights[vocab2.getIds(['president'])])

[15]
tf.Tensor(
[ 0.13823049  0.48260826  0.3182307  -0.212288   -0.46534726 -0.0651701
  0.5990064   0.08913989 -0.07262058  0.5766061   0.7129216  -0.70614886
 -0.24771981 -0.41767475 -0.4930174  -0.5288774   0.24726069 -0.70448744
  0.6120768  -0.4392945  -0.41103184 -0.19654481  0.24717218  0.5823086
  0.27910572  0.5151664   0.25357673  0.59487754 -0.47175306  0.11838285
  0.0909834  -0.7501625  -0.18798919 -0.70995975 -0.67468905  0.60312814
 -0.06865751  0.5032709   0.03920567  0.40931952  0.09880754 -0.09851276
 -0.26098204  0.186985   -0.33710906  0.05847093 -0.03381895  0.36705905
  0.05159425  0.7304324   0.00723518  0.3679483   0.33992016  0.22369757
  0.22721064  0.2475165   0.45047453 -0.7886953   0.6865816  -0.5037599
  0.6836924   0.54743636  0.11336728 -0.59786993  0.652192   -0.32566872
 -0.22432154  0.48325863  0.07317996 -0.73269224  0.490839   -0.47635692
  0.27007115 -0.5615491   0.30989116  0.2723256  -0.61011606 -0.0110365
  0.62017024 -0.50812     0.5791151  -0

In [11]:
pip install cudnn

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement cudnn (from versions: none)
ERROR: No matching distribution found for cudnn
